<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=261b3ed601cfc866e2bdef689aef7b3ba40e3260ca860c5206f2212c4978f6e4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 11:45:36
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -17.63 K (-0.12%)
Current PnL: -20.59 L (-13.66%)
CY Booked + Current PnL: -9.15 L (-6.07%)
-------------------
Total profit:  1.46 L
Total loss:  -22.05 L
-------------------
Total Booked + Current PnL: 17.83 L (14.42%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.09 L (62.48%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115376.0,14872.0,14237.0,0.25,14.80,12.34,28.96,79.0,1.04,0.82,53.79,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.01,53.0,H-MC,2.99,191590.0,26920.0,16419.0,1.22,16.35,8.57,26.31,89.0,1.64,1.36,25.64,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.35,83329.0,-17448.0,17541.0,0.11,-17.31,21.05,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207541.0,3041.0,20256.0,0.12,1.49,9.76,11.39,10.0,0.15,1.48,10.57,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.0,14.89,66.0,H-SC,5.81,256170.0,-24953.0,157826.0,2.93,-8.88,61.61,47.26,144.0,-0.16,1.82,37.25,XY24,NTT,INSURANCE
85,WHIRLPOOL,2270.0,-46.22,36.0,M-SC,1.08,90892.0,-606.0,79358.0,2.42,-0.66,87.31,86.07,223.0,-0.01,0.65,31.99,XR,NTT,DURABLES
44,JIOFIN,387.0,-1.15,54.0,H-LC,12.75,268664.0,1299.0,64157.0,2.01,0.49,23.88,24.48,48.0,0.02,1.91,55.73,XY24,BTT,FINANCE
26,GREENPANEL,537.0,235.27,61.0,M-SC,3.71,149816.0,-33262.0,112242.0,1.59,-18.17,74.92,43.14,230.0,-0.30,1.07,40.92,XY24,NTT,MISC
39,INDUSINDBK,1800.0,248.74,51.0,L-MC,11.86,42784.0,-34422.0,59816.0,1.53,-44.58,139.81,32.89,258.0,-0.58,0.30,17.82,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151164.0,-27907.0,93631.0,-2.50,-15.58,61.94,36.71,135.0,-0.30,1.08,49.98,XR,NTT,BANKS
64,SHALBY,327.00,1218.98,63.0,M-SC,20.82,179279.0,-1864.0,47330.0,-2.38,-1.03,26.40,25.10,235.0,-0.04,1.28,43.67,XY24,NTT,HEALTHCARE
38,INDIGOPNTS,1408.00,100.33,35.0,M-SC,2.11,131514.0,-43045.0,43084.0,-1.91,-24.66,32.76,0.02,221.0,-1.00,0.94,14.39,OX40N,NTT,PAINTS
46,KPIGREEN,731.05,-7.87,30.0,H-SC,6.76,109872.0,-15425.0,74350.0,-1.58,-12.31,67.67,47.03,141.0,-0.21,0.78,37.57,MH,ATH,POWER
73,TATAMOTORS,1065.00,-19.53,52.0,M-LC,2.29,254989.0,-21534.0,131600.0,-1.43,-7.79,51.61,39.81,54.0,-0.16,1.82,22.21,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-46.22,36.0,M-SC,1.08,90892.0,-606.0,79358.0,2.42,-0.66,87.31,86.07,223.0,-0.01,0.65,31.99,XR,NTT,DURABLES
60,SAIL,228.0,43.54,51.0,M-MC,12.34,227011.0,2049.0,165605.0,-0.60,0.91,72.95,74.53,192.0,0.01,1.62,33.46,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.00,49.0,H-MC,2.06,136018.0,230.0,42519.0,0.23,0.17,31.26,31.48,107.0,0.01,0.97,12.31,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,44.0,M-LC,5.71,150424.0,-521.0,109975.0,0.20,-0.35,73.11,72.51,53.0,-0.00,1.07,5.66,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,65.0,H-SC,13.51,131166.0,-11313.0,30837.0,-0.63,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,55.0,H-SC,2.11,224550.0,-45117.0,81444.0,-0.59,-16.73,36.27,13.47,138.0,-0.55,1.60,14.18,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.35,83329.0,-17448.0,17541.0,0.11,-17.31,21.05,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,38.0,H-SC,1.52,140818.0,-35085.0,77112.0,-0.01,-19.95,54.76,23.89,149.0,-0.45,1.00,21.61,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.05,40.0,H-MC,6.27,101318.0,-30517.0,72746.0,-1.14,-23.15,71.80,32.03,98.0,-0.42,0.72,13.48,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,51.0,M-MC,12.34,227011.0,2049.0,165605.0,-0.60,0.91,72.95,74.53,192.0,0.01,1.62,33.46,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211671.0,6595.0,105984.0,-1.00,3.22,50.07,54.89,52.0,0.06,1.51,28.09,X5K,ATH,METALS
14,BLUESTARCO,2080.00,11.01,53.0,H-MC,2.99,191590.0,26920.0,16419.0,1.22,16.35,8.57,26.31,89.0,1.64,1.36,25.64,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,49.0,H-MC,1.43,205890.0,14148.0,23616.0,0.43,7.38,11.47,19.69,99.0,0.60,1.47,14.68,XY25,NTT,AC
17,CAMS,5211.76,-5.38,48.0,H-SC,1.81,107685.0,5681.0,38239.0,0.53,5.57,35.51,43.06,122.0,0.15,0.77,23.64,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115376.0,14872.0,14237.0,0.25,14.80,12.34,28.96,79.0,1.04,0.82,53.79,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194480.0,14672.0,97746.0,0.33,8.16,50.26,62.52,88.0,0.15,1.38,45.03,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211671.0,6595.0,105984.0,-1.00,3.22,50.07,54.89,52.0,0.06,1.51,28.09,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,36.0,H-SC,9.35,126092.0,2756.0,128866.0,0.13,2.23,102.20,106.72,119.0,0.02,0.90,26.14,AR,ATH,MISC
86,WIPRO,420.00,-14.25,44.0,M-LC,5.71,150424.0,-521.0,109975.0,0.20,-0.35,73.11,72.51,53.0,-0.00,1.07,5.66,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.45,29.0,H-LC,9.97,279061.0,-36781.0,149437.0,-0.35,-11.65,53.55,35.67,5.0,-0.25,1.99,0.60,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.77,207055.0,-44713.0,97916.0,-0.08,-17.76,47.29,21.13,27.0,-0.46,1.47,11.37,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-10.50,41.0,H-LC,8.00,251420.0,-5643.0,40780.0,-1.09,-2.20,16.22,13.67,24.0,-0.14,1.79,17.77,XY25,NTT,FMCG
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195967.0,-4171.0,25064.0,-0.88,-2.08,12.79,10.44,4.0,-0.17,1.40,3.36,X40,NTT,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278531.0,-67425.0,135672.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.59,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.17,54.0,H-LC,1.98,161150.0,-24945.0,72421.0,-0.88,-13.40,44.94,25.51,15.0,-0.34,1.15,18.76,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,42.0,H-LC,2.04,195967.0,-4171.0,25064.0,-0.88,-2.08,12.79,10.44,4.0,-0.17,1.40,3.36,X40,NTT,FMCG
48,LTIM,7230.2,-4.05,54.0,H-LC,2.37,237622.0,-8529.0,87730.0,0.12,-3.47,36.92,32.18,16.0,-0.10,1.69,31.33,X200,ATH,IT
51,NESTLEIND,1377.0,-10.13,52.0,H-LC,3.05,276377.0,10951.0,45851.0,-0.05,4.13,16.59,21.40,11.0,0.24,1.97,11.04,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.45,29.0,H-LC,9.97,279061.0,-36781.0,149437.0,-0.35,-11.65,53.55,35.67,5.0,-0.25,1.99,0.60,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278531.0,-67425.0,135672.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.59,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195967.0,-4171.0,25064.0,-0.88,-2.08,12.79,10.44,4.0,-0.17,1.40,3.36,X40,NTT,FMCG
30,HCLTECH,1937.36,-1.94,48.0,H-LC,9.88,224180.0,-17744.0,79987.0,0.72,-7.33,35.68,25.73,8.0,-0.22,1.60,5.94,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.09,311214.0,-1674.0,173346.0,-1.01,-0.54,55.70,54.87,3.0,-0.01,2.22,6.08,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,54.0,H-LC,1.98,161150.0,-24945.0,72421.0,-0.88,-13.40,44.94,25.51,15.0,-0.34,1.15,18.76,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195967.0,-4171.0,25064.0,-0.88,-2.08,12.79,10.44,4.0,-0.17,1.40,3.36,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.77,207055.0,-44713.0,97916.0,-0.08,-17.76,47.29,21.13,27.0,-0.46,1.47,11.37,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207541.0,3041.0,20256.0,0.12,1.49,9.76,11.39,10.0,0.15,1.48,10.57,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215670.0,5304.0,23486.0,0.55,2.52,10.89,13.68,37.0,0.23,1.54,19.46,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278531.0,-67425.0,135672.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.59,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.09,311214.0,-1674.0,173346.0,-1.01,-0.54,55.70,54.87,3.0,-0.01,2.22,6.08,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.04,195967.0,-4171.0,25064.0,-0.88,-2.08,12.79,10.44,4.0,-0.17,1.40,3.36,X40,NTT,FMCG
83,VBL,671.64,-22.45,29.0,H-LC,9.97,279061.0,-36781.0,149437.0,-0.35,-11.65,53.55,35.67,5.0,-0.25,1.99,0.60,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,53.0,H-LC,7.50,252792.0,-8827.0,128039.0,0.93,-3.37,50.65,45.57,7.0,-0.07,1.80,7.57,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,31.59,78720.0,-34829.0,74878.0,-0.56,-30.67,95.12,35.27,268.0,-0.47,0.56,92.67,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,44.0,H-SC,11.44,91560.0,-6420.0,28027.0,-0.96,-6.55,30.61,22.05,152.0,-0.23,0.65,33.09,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115376.0,14872.0,14237.0,0.25,14.80,12.34,28.96,79.0,1.04,0.82,53.79,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,65.0,H-SC,13.51,131166.0,-11313.0,30837.0,-0.63,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145809.0,-3411.0,147588.0,1.29,-2.29,101.22,96.62,208.0,-0.02,1.04,57.80,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145809.0,-3411.0,147588.0,1.29,-2.29,101.22,96.62,208.0,-0.02,1.04,57.80,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151164.0,-27907.0,93631.0,-2.50,-15.58,61.94,36.71,135.0,-0.30,1.08,49.98,XR,NTT,BANKS
26,GREENPANEL,537.00,235.27,61.0,M-SC,3.71,149816.0,-33262.0,112242.0,1.59,-18.17,74.92,43.14,230.0,-0.30,1.07,40.92,XY24,NTT,MISC
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194480.0,14672.0,97746.0,0.33,8.16,50.26,62.52,88.0,0.15,1.38,45.03,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,64.0,M-LC,6.19,211671.0,6595.0,105984.0,-1.00,3.22,50.07,54.89,52.0,0.06,1.51,28.09,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.10
1,25,43.29
2,50,75.03


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.12
LC    32.17
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.59
X40N     11.20
XR       11.02
XY25     10.42
AR        8.21
OX40N     7.71
X200      1.69
MH        1.60
X5K       1.51
SR        1.14
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.00
H-LC    25.64
H-MC    20.53
M-SC    13.70
M-LC     5.51
M-MC     2.93
L-SC     1.42
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-6.19,41.19
IT,12.61,-18.73,83.04
FINANCE,10.49,-14.62,69.98
MISC,6.98,-17.61,82.77
BANKS,6.37,-11.27,69.80
PAINTS,5.55,-18.47,36.13
ELECTRICAL,5.38,-8.95,47.18
INSURANCE,3.81,-3.32,41.95
AC,3.52,2.05,14.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229194.0,22
XR,1364454.0,14
AR,1177932.0,9
X40,792222.0,10
X40N,662115.0,9
OX40N,563465.0,10
XY25,476303.0,7
SR,261506.0,2
X5K,105984.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3721273.0,29
M-SC,1511421.0,17
H-MC,1270458.0,15
H-LC,1194579.0,15
M-LC,376505.0,4
M-MC,371918.0,2
L-SC,261947.0,3
L-MC,59816.0,1
L-LC,41575.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1211577.0      6
M-SC       XY24       851678.0      7
H-SC       AR         808948.0      5
           XR         796271.0      7
H-MC       XY24       569318.0      4
H-LC       X40        511985.0      5
M-MC       XY24       371918.0      2
H-SC       X40N       310197.0      4
           SR         261506.0      2
           OX40N      258424.0      4
H-LC       X40N       220130.0      3
H-MC       X40        207600.0      4
H-LC       AR         200460.0      2
H-MC       XY25       177023.0      2
L-SC       XR         172779.0      2
M-SC       AR         168524.0      2
M-LC       XY24       160546.0      2
M-SC       XY25       147588.0      1
           OX40N      143127.0      4
H-MC       X40N       131788.0      2
M-SC       XR         127867.0      2
H-LC       XY25       110117.0      3
M-LC       XR         109975.0      1
           X5K        105984.0      1
H-MC       XR          97746.0      1
L-SC       OX40N       89168.0      1
H-LC       X200        87730.0      1
H-SC       MH          74350.0      1
H-MC       OX40N       72746.0      1
M-SC       X40         72637.0      1
H-LC       XY24        64157.0      1
L-MC       XR          59816.0      1
L-LC       XY25        41575.0      1
H-MC       MH          14237.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
